In [ ]:
import pandas as pd
import model as m
import pymc3 as pm
import model_helper as mh
import theano.tensor as tt
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns_c = sns.color_palette(palette='deep')

In [ ]:
start_date = pd.to_datetime('2020-07-01')
end_date = pd.to_datetime('2020-12-01')

# Cases
cases = pd.read_csv("data/casos_tecnica_ccaa.csv")
cases = cases.loc[cases["ccaa_iso"] == "AR"]
cases["fecha"] = pd.to_datetime(cases["fecha"])
cases.set_index("fecha", inplace=True)
cases = cases["num_casos"]
cases_train = cases.loc[start_date:end_date]

# Hospitalizaded
hospitalized = pd.read_csv("data/casos_hosp_uci_def_sexo_edad_provres.csv")
hospitalized = hospitalized[hospitalized.provincia_iso.isin(["Z","HU","TE"])]
hospitalized = hospitalized.groupby("fecha").sum()
hospitalized.index = pd.to_datetime(hospitalized.index)
hospitalized = hospitalized["num_hosp"]
hospitalized_train = hospitalized.loc[start_date:end_date]

In [ ]:
# Run model
draws = 5000
burn = 2000
with m.admissions_model(cases_train, hospitalized_train) as model:
    trace = pm.sample(draws, tune=burn)

In [ ]:
import arviz as az
az.summary(trace,hdi_prob=.95)

In [ ]:
az.plot_trace(trace)

In [ ]:
with model:
    admissions = pm.sample_posterior_predictive(trace,10000)

In [ ]:
dates = pd.date_range(start_date, end_date).strftime("%m-%d")
plot_dates = [dates[i] for i in range(0, len(hospitalized_train), 21)]

posterior_quantile = np.percentile(admissions["admissions"], [2.5, 25, 50, 75, 97.5], axis=0)

# Plot daily number of admissions
plt.plot(
    dates, posterior_quantile[2, :],
    color='b', label='posterior median', lw=2)

plt.fill_between(
    dates, posterior_quantile[0, :], posterior_quantile[4, :],
    color='b', label='95% quantile', alpha=.2)

plt.plot(
      dates, hospitalized_train,
      '--o', color='k', markersize=3,
      label='Observed admissions', alpha=.8)

plt.xticks(plot_dates)
plt.ylabel('Daily number of admissions', fontsize='large')
plt.xlabel('Day', fontsize='large')

fontsize = 'medium'
plt.legend(loc='upper left', fontsize=fontsize)

plt.tight_layout();

In [ ]:
end_date = pd.to_datetime('2021-07-12')
cases_test = cases.loc[start_date:end_date]
hospitalized_test = hospitalized.loc[start_date:end_date]

with m.admissions_model(cases_test, np.empty(cases_test.shape)) as predictive_model:
    prediction = pm.sample_posterior_predictive(trace, samples=5000)

In [ ]:
dates = pd.date_range(start_date, end_date).strftime("%y-%m-%d")
plot_dates = [dates[i] for i in range(0, len(hospitalized_test), 28)]

posterior_quantile = np.percentile(prediction["admissions"], [2.5, 25, 50, 75, 97.5], axis=0)

# Plot daily number of admissions
plt.plot(
    dates, posterior_quantile[2, :],
    color='b', label='posterior median', lw=2)

plt.fill_between(
    dates, posterior_quantile[0, :], posterior_quantile[4, :],
    color='b', label='95% quantile', alpha=.2)

plt.plot(
      dates, hospitalized_test,
      '--o', color='k', markersize=3,
      label='Observed admissions', alpha=.8)

plt.xticks(plot_dates)
plt.ylabel('Daily number of admissions', fontsize='large')
plt.xlabel('Day', fontsize='large')

fontsize = 'medium'
plt.legend(loc='upper right', fontsize=fontsize)

plt.tight_layout();

# Output

In [ ]:
np.savetxt("results/pH_admission_AR.txt", trace["pH", :])
np.savetxt("results/beta_admission_AR.txt", trace["admissions_beta", :])
np.savetxt("results/sigma_admission_AR.txt", trace["sigma", :])

data = {"median" : posterior_quantile[2, :],
        "lower" : posterior_quantile[0, :],
        "upper" : posterior_quantile[4, :],
        "observed" : hospitalized_test
       }

pd.DataFrame.from_dict(data).to_csv("results/values_admission_AR.csv")